In [230]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import string
import math
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import warnings
from nltk.stem import WordNetLemmatizer  
from nltk.tokenize import word_tokenize 
warnings.simplefilter("ignore", category=PendingDeprecationWarning)

In [231]:
dfUsers = pd.read_csv('Users.csv')
dfEvents = pd.read_csv('Events.csv')

In [234]:
def removeStopwords(sentence):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence)   
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return ' '.join(filtered_sentence)

In [235]:
def removePunctuations(sentence):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in sentence:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [236]:
def extractCompanyTags(para):
    df_Company_csv = pd.read_csv('Companies.csv')
    df_Companies = pd.DataFrame(columns = ['Company'])
    df_Companies['Company'] = df_Company_csv['Company']
    df_Companies['Company'] = df_Companies['Company'].astype(str).str.lower()
    filteredTags = []
    for index,row in df_Companies.iterrows():
        if row['Company'].lower() in para.lower():
            filteredTags.append(row['Company'].lower())
    return filteredTags

In [237]:
def extractKeywords(para):
    df_keywords_csv = pd.read_csv('Tech words.csv')
    df_keywords_csv['Keys'] = df_keywords_csv['Keys'].astype(str).str.lower()
    filteredTags = []
    for index,row in df_keywords_csv.iterrows():
        if row['Keys'] in para.lower():
            filteredTags.append(row['Keys'])
    return filteredTags

In [238]:
def UserCleaning(dfUsers):
    filtered_sentences = []
    for i in range(len(dfUsers.index)):
        sent = dfUsers.loc[i]['Jobs Applied']
        list3 = []
        if(type(sent)!=str):
            list3 = list3
        else:
            sent = sent.lower()
            sent = removeStopwords(sent)
            sent = removePunctuations(sent)
            list1 = extractCompanyTags(sent)
            list2 = extractKeywords(sent)
            list3 = list1 + list2
        sent = dfUsers.loc[i]['Job Experience']
        if(type(sent)!=str):
            list3 = list3
        else:
            sent = sent.lower()
            sent = removeStopwords(sent)
            sent = removePunctuations(sent)
            list1 = extractCompanyTags(sent)
            list2 = extractKeywords(sent)
            list3 = list3 + list1 + list2
        list3 = list3 + extractKeywords(dfUsers.loc[i]['Skills'])
        filtered_sentences.append(list3)
    dfUsers['Tags'] = filtered_sentences
    return dfUsers

In [239]:
def EventCleaning(dfEvents):
    filtered_sentences = []
    for i in range(len(dfEvents.index)):
        sent = dfEvents.loc[i]['Description']
        if(type(sent)!=str):
            list3 = []
        else:
            sent = sent.lower()
            sent = removeStopwords(sent)
            sent = removePunctuations(sent)
            list1 = extractCompanyTags(sent)
            list2 = extractKeywords(sent)
            list3 = list1 + list2
        list3 = list3 + extractKeywords(dfEvents.loc[i]['Skills'])
        filtered_sentences.append(list3)
    dfEvents['Tags'] = filtered_sentences
    return dfEvents

In [240]:
dfUsers = UserCleaning(dfUsers)

In [241]:
dfEvents = EventCleaning(dfEvents)

In [244]:
def userTags(user,dfUsers):
    listUserTags = list()
    for index,row in dfUsers.iterrows():
        if row['User Name'] == user:
            return list(set(row['Tags']))

def calculateScore(userTags,df):
    scores = []
    for index,row in df.iterrows():
        eventTags = list(set(row['Tags']))
        common = retCommon(eventTags, userTags)
        scores.append(len(common)/(len(userTags)))
    sv = pd.Series(scores)
    df['scores'] = sv.values
    return df

def retCommon(list1,list2):
    common = []
    for ele in list2:
        if ele in list1:
            common.append(ele)
    common = list(set(common))
    return common

calculateScore(userTags('Alex',dfUsers),dfEvents)

['sql', 'r', 'cyber', 'c', 'ip', 'python', 'hacking', 'ca', 'html']
['python', 'c', 'r']
['r', 'c', 'ca']
['c', 'r']
['python', 'r', 'c', 'ca']
['c', 'r']
['c', 'r']
['c', 'r']
[0.42857142857142855, 0.14285714285714285, 0.14285714285714285, 0.09523809523809523, 0.19047619047619047, 0.09523809523809523, 0.09523809523809523, 0.09523809523809523]


,Event Id,Event Name,Description,Speakers,Skills,Audience,Venue,Website,Date,Tags,Scores,scores
0,1,Nullcon Conference,Nullcon was founded in 2010 with the idea of p...,"Vikas Gupta, Nachiket Trivedi, Shila Trivedi, ...","CTF, Hacking, IoT, Python, JailBeak, Web Hacki...","Security Practitioners, Secuirty Analysts, Sec...","Goa, India",https://nullcon.net,01/03/2019 - 02/03/2019,"[ca, next, oi, hacking, c, r, c, hacking, pyth...",0.400668,0.428571
1,2,Pycon India,"PyCon India, the premier conference in India o...","Alex, Rima, Vikas Gupta","Python, Python Programming, PyTorch, Artificia...","Open Source, Data Scienctist, Hacker, Student,...","Hyderabad, Karnataka, India",https://in.pycon.org,05/10/2019 - 10/10/2019,"[premier, python, python, python developer, pr...",0.160128,0.142857
2,3,4th International Conference on Magnetism and ...,Conference Series llc LTD heartily welcomes yo...,"Harsh Bhatt, Yogesh Tirpathi, Fevin Balat, Var...","Magnetism, Material Science, Electromagnetism,...","Physicists, Students, Eminent Scientists from ...","London, UK",https://magneticmaterials.physicsmeeting.com,01/02/2019 - 02/02/2019,"[ubs, sse, ppl, ses, terna, ca, iss, oi, c, r,...",0.000000,0.142857
3,4,Computer Vision Meetup,Image processing - a buzz word in today's worl...,NaN,"Image Processing, computer Vision, Object dete...","Researchers, Reviewers, BTech students, Comput...","Banglore, Karnataka, India",www.google.com,06/02/2019 - 07/02/2019,"[ses, computer vision, image processing, c, r,...",NaN,0.095238
4,5,PyData Meetup,As can be seen Python is becoming one of the l...,NaN,"Python, Python Programming, PyTorch, Artificia...","College Students, Btech, mtech, mscit, masters...","Gandhinagar, Gujarat, India",www.google.com,17/10/2019 - 18/10/2019,"[ppl, lear, ca, python, natural language proce...",NaN,0.190476
5,6,Cloud Computing MeetUp #6,An event describing various cloud computing pl...,NaN,"Cloud Computing, AMazon Web services, AWS, Goo...",NaN,"Delhi, India",www.google.com,01/12/2019 - 02/12/2019,"[c, r, cloud computing, c, cloud computing, c,...",NaN,0.095238
6,7,Femina Miss India Contest,A beauty event where models from different pla...,"Fashion, Beauty, Beautiful, Pretty, Compact, M...","Cat Walk, Fashion Background, Beauty Paegant",NaN,"Mumbai, Maharashtra, India",www.google.com,03/05/2019 - 04/05/2019,"[c, r, c, c, r, c]",NaN,0.095238
7,8,Cooking Culture,A cooking competition among various cooking ex...,"Cook, Chef, Kitchen, Dishes","Cooking, Mixing, Predicting, Tasty Food Prepar...",NaN,"Hyderabad, Karnataka, India",www.google.com,06/11/2019 - 07/11/2019,"[c, r, c, c, r, c]",NaN,0.095238
